In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from psifr import fr
import swarmbar as sb

from cymr import cmr
from dcmr import framework
from dcmr import figures
from dcmr import task
from dcmr import reports

# res_dir = Path(os.environ['CDC_RESULTS'])
res_dir = Path('/Users/morton/Dropbox/work/dcmr/cdcatfr2')
# fit_dir = Path(os.environ['CDC_FITS'])
fit_dir = Path('/Users/morton/Dropbox/work/dcmr/cdcatfr2/fits')
# fig_dir = Path(os.environ['CDC_FIGURES'])
fig_dir = Path('/Users/morton/Dropbox/work/dcmr/cdcatfr2/figs/v1') / 'plots'

data_file = res_dir / 'data.csv'
patterns_file = res_dir / 'patterns.hdf5'
model_dir = fit_dir / 'v1'
fig_dir.mkdir(exist_ok=True, parents=True)
figures.set_style()

In [2]:
patterns = cmr.load_patterns(patterns_file)
data = pd.read_csv(data_file).astype({'item_index': 'Int64'})
data.head()

,subject,list,position,trial_type,item,category,distractor,item_index
0,201,1,1,study,MASCARA,obj,0.0,646
1,201,1,2,study,GOLF CLUB,obj,0.0,606
2,201,1,3,study,KEYCHAIN,obj,0.0,626
3,201,1,4,study,LOS ANGELES,loc,0.0,370
4,201,1,5,study,GETTYSBURG,loc,0.0,325


In [3]:
data.shape

(17793, 8)

In [4]:
models = [
    'v1',
]
model_names = [
    'DCMR-Disrupt',
]
data = framework.read_model_sims(data_file, fit_dir, models, model_names, data_first=True)
data['Distraction (s)'] = data['distractor']
data.head()

subject  list         item  input  output  study  recall  \
source trial                                                             
Data   0          201     1      MASCARA      1    <NA>   True   False   
       1          201     1    GOLF CLUB      2    <NA>   True   False   
       2          201     1     KEYCHAIN      3    <NA>   True   False   
       3          201     1  LOS ANGELES      4    <NA>   True   False   
       4          201     1   GETTYSBURG      5       9   True    True   

              repeat  intrusion  distractor  item_index category  \
source trial                                                       
Data   0           0      False         0.0         646      obj   
       1           0      False         0.0         606      obj   
       2           0      False         0.0         626      obj   
       3           0      False         0.0         370      loc   
       4           0      False         0.0         325      loc   

              Distraction (s)  
source trial                   
Data   0                  0.0  
       1                  0.0  
       2                  0.0  
       3                  0.0  
       4                  0.0

In [5]:
curve = sns.color_palette(
    'ch:rot=-.5, light=.6, dark=.3, gamma=1', n_colors=3
)
curve

[(0.44163916703794076, 0.6914342033666265, 0.5414714890673176),
 (0.27424883716016524, 0.5352555384607786, 0.4809527643729599),
 (0.17051215858853222, 0.344636926092434, 0.3959064384004337)]

In [6]:
dark = sns.color_palette(
    'ch:rot=-.5, light=.7, dark=.3, gamma=.6', n_colors=3
)
dark

[(0.725301473503126, 0.8645990769601404, 0.7363315007817791),
 (0.5016677447743987, 0.7426793533080586, 0.6602162691374847),
 (0.33145894574681245, 0.5393097642399239, 0.6005096167923277)]

In [7]:
light = sns.color_palette(
    'ch:rot=-.5, light=.7, dark=.3, gamma=.2', n_colors=3
)
light

[(0.8973525746116884, 0.9546730560357413, 0.901891392639006),
 (0.7912737188119296, 0.9119907090651371, 0.870686908851388),
 (0.6820851323976215, 0.8225675908928259, 0.8639314200489312)]

In [8]:
spc = data.groupby(['source', 'Distraction (s)'], sort=False).apply(fr.spc)
g = fr.plot_spc(spc, hue='Distraction (s)', col='source', palette=curve, height=5).add_legend().set_titles('{col_name}')
g.savefig(fig_dir / 'spc.pdf')

/var/folders/4k/wss4rkc16j70t_mk8f3vf7300000gn/T/ipykernel_21509/1702233231.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spc = data.groupby(['source', 'Distraction (s)'], sort=False).apply(fr.spc)
LINO NOT subset; don't know how to subset; dropped
LINO NOT subset; don't know how to subset; dropped


In [9]:
pfr = data.groupby(['source', 'Distraction (s)'], sort=False).apply(fr.pnr).query('output == 1')
g = fr.plot_spc(pfr, hue='Distraction (s)', col='source', palette=curve, height=5).add_legend().set_titles('{col_name}')
g.savefig(fig_dir / 'pfr.pdf')

/var/folders/4k/wss4rkc16j70t_mk8f3vf7300000gn/T/ipykernel_21509/23851514.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pfr = data.groupby(['source', 'Distraction (s)'], sort=False).apply(fr.pnr).query('output == 1')
LINO NOT subset; don't know how to subset; dropped
LINO NOT subset; don't know how to subset; dropped


In [10]:
kwargs = {'dark': dark, 'light': light, 'point_kws': {'size': 5}}

In [11]:
clust = data.groupby(['source', 'distractor']).apply(fr.category_clustering, category_key='category')
clust.groupby(['source', 'distractor']).agg(['mean', 'sem'])

/var/folders/4k/wss4rkc16j70t_mk8f3vf7300000gn/T/ipykernel_21509/1582668125.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clust = data.groupby(['source', 'distractor']).apply(fr.category_clustering, category_key='category')


subject            lbc                 arc          
                           mean  sem      mean       sem      mean       sem
source       distractor                                                     
DCMR-Disrupt 0.0          205.9  1.1  2.028319  0.145603  0.463912  0.018175
             2.5          205.9  1.1  1.474628  0.130932  0.410032  0.029988
             7.5          205.9  1.1  1.348290  0.117578  0.396589  0.030418
Data         0.0          205.9  1.1  2.180676  0.138734  0.531257  0.027473
             2.5          205.9  1.1  1.422464  0.145531  0.408586  0.049215
             7.5          205.9  1.1  1.465459  0.164687  0.495812  0.042237

In [12]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
sb.plot_swarm_bar(clust.loc['Data'], x='distractor', y='lbc', hue='distractor', legend=False, ax=ax[0], **kwargs);
ax[0].set_ylabel('LBC')
ax[0].set_ylim(0, 3)
ax[0].set_xlabel('Distraction (s)')
ax[0].set_title('Data')
ax[0].tick_params(axis='x', labelsize='small')
sb.plot_swarm_bar(clust.loc['DCMR-Disrupt'], x='distractor', y='lbc', hue='distractor', legend=False, ax=ax[1], **kwargs);
ax[1].set_xlabel('Distraction (s)')
ax[1].set_title('DCMR-Disrupt')
ax[1].tick_params(axis='x', labelsize='small')
fig.savefig(fig_dir / 'lbc.pdf')

LINO NOT subset; don't know how to subset; dropped
LINO NOT subset; don't know how to subset; dropped


In [13]:
temp = data.groupby(['source', 'distractor']).apply(fr.lag_rank, test_key='category', test=lambda x, y: x == y)
temp.groupby(['source', 'distractor']).agg(['mean', 'sem'])

/var/folders/4k/wss4rkc16j70t_mk8f3vf7300000gn/T/ipykernel_21509/1277313943.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = data.groupby(['source', 'distractor']).apply(fr.lag_rank, test_key='category', test=lambda x, y: x == y)


subject           rank          
                           mean  sem      mean       sem
source       distractor                                 
DCMR-Disrupt 0.0          205.9  1.1  0.661610  0.004874
             2.5          205.9  1.1  0.669681  0.012261
             7.5          205.9  1.1  0.658806  0.012077
Data         0.0          205.9  1.1  0.676020  0.008431
             2.5          205.9  1.1  0.664947  0.023335
             7.5          205.9  1.1  0.679256  0.014973

In [14]:
p = clust.loc['Data', 'lbc'].unstack().T
pg.ttest(p[0.0], p[2.5], paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.176331,9,two-sided,0.002389,"[0.35, 1.17]",1.686451,19.854,0.996984


In [15]:
p = clust.loc['DCMR-Disrupt', 'lbc'].unstack().T
pg.ttest(p[0.0], p[2.5], paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.103368,9,two-sided,0.002663,"[0.25, 0.86]",1.264549,18.156,0.943605


In [16]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), sharey=True)
sb.plot_swarm_bar(temp.loc['Data'], x='distractor', y='rank', hue='distractor', legend=False, ax=ax[0], **kwargs);
ax[0].set_ylabel('Temporal clustering')
ax[0].set_xlabel('Distraction (s)')
ax[0].set_title('Data')
ax[0].tick_params(axis='x', labelsize='small')
ax[0].set_ylim(0, 1)
sb.plot_swarm_bar(temp.loc['DCMR-Disrupt'], x='distractor', y='rank', hue='distractor', legend=False, ax=ax[1], **kwargs);
ax[1].set_xlabel('Distraction (s)')
ax[1].set_title('DCMR-Disrupt')
ax[1].tick_params(axis='x', labelsize='small')
fig.savefig(fig_dir / 'tempclust.pdf')

LINO NOT subset; don't know how to subset; dropped
LINO NOT subset; don't know how to subset; dropped


In [17]:
p = temp.loc['Data', 'rank'].unstack().T
pg.ttest(p[0.0], p[2.5], paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.491749,9,two-sided,0.634668,"[-0.04, 0.06]",0.199598,0.342,0.087504


In [18]:
p = temp.loc['DCMR-Disrupt', 'rank'].unstack().T
pg.ttest(p[0.0], p[2.5], paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.644432,9,two-sided,0.535373,"[-0.04, 0.02]",0.273546,0.368,0.121307


In [19]:
table, raw = reports.create_model_table(fit_dir, models, model_names, model_comp=None)
table.to_latex(fig_dir / 'parameters_cdcatfr2.tex', escape=False)
table

,DCMR-Disrupt
$\phi_s$,0.05 (0.01)
$\phi_d$,2.03 (0.59)
"$\beta_{\mathrm{enc},I}$",0.56 (0.02)
"$\beta_{\mathrm{enc},C}$",0.08 (0.04)
"$\beta_{\mathrm{enc},D}$",0.48 (0.12)
"$\beta_{\mathrm{rec},I}$",0.67 (0.02)
$w_1$,0.85 (0.03)
$w_2$,0.15 (0.03)
$\beta_{\mathrm{disrupt}}$,0.48 (0.14)
"$\beta_{\mathrm{start},\mathrm{0 s}}$",0.45 (0.12)
